# Topic 8: RASA Chatbot - SOLUTIONS

Complete solutions for creating a RASA chatbot with proper training data, domain configuration, and conversation flows.

In [ ]:
# Essential imports
from pathlib import Path
import yaml
import json
import os
import warnings
warnings.filterwarnings('ignore')

print("RASA Chatbot Solutions")
print("=" * 40)

# Check if RASA is available
try:
    import rasa
    RASA_AVAILABLE = True
    print(f"✓ RASA version {rasa.__version__} available!")
except ImportError:
    RASA_AVAILABLE = False
    print("⚠ RASA not available. Install with: pip install rasa")

print()

## Solution 1: Project Structure Setup

In [ ]:
def create_rasa_project_structure():
    """Create a complete RASA project structure."""
    
    PROJECT_ROOT = Path.cwd()
    RASA_DIR = PROJECT_ROOT / 'rasa_project'
    
    # Create directory structure
    directories = [
        'data',
        'actions',
        'models',
        'tests',
        'config'
    ]
    
    for directory in directories:
        (RASA_DIR / directory).mkdir(parents=True, exist_ok=True)
    
    print("Created RASA project structure:")
    print(f"Project root: {RASA_DIR}")
    
    for directory in directories:
        print(f"  📁 {directory}/")
    
    return RASA_DIR

def create_domain_file(rasa_dir):
    """Create a comprehensive domain.yml file."""
    
    domain_config = {
        'version': '3.1',
        'intents': [
            'greet',
            'goodbye',
            'affirm',
            'deny',
            'mood_great',
            'mood_unhappy',
            'bot_challenge',
            'ask_opening_hours',
            'ask_contact_info',
            'ask_services',
            'book_appointment',
            'cancel_appointment',
            'ask_weather',
            'ask_time',
            'ask_help'
        ],
        'entities': [
            'time',
            'date',
            'person',
            'location',
            'service_type',
            'appointment_type'
        ],
        'slots': {
            'appointment_date': {
                'type': 'text',
                'mappings': [
                    {'type': 'from_entity', 'entity': 'date'}
                ]
            },
            'appointment_time': {
                'type': 'text',
                'mappings': [
                    {'type': 'from_entity', 'entity': 'time'}
                ]
            },
            'service_type': {
                'type': 'text',
                'mappings': [
                    {'type': 'from_entity', 'entity': 'service_type'}
                ]
            },
            'user_name': {
                'type': 'text',
                'mappings': [
                    {'type': 'from_entity', 'entity': 'person'}
                ]
            }
        },
        'responses': {
            'utter_greet': [
                {'text': 'Hallo! Wie kann ich Ihnen helfen?'},
                {'text': 'Guten Tag! Womit kann ich Ihnen behilflich sein?'},
                {'text': 'Willkommen! Wie kann ich Sie unterstützen?'}
            ],
            'utter_goodbye': [
                {'text': 'Auf Wiedersehen!'},
                {'text': 'Tschüss! Haben Sie einen schönen Tag!'},
                {'text': 'Bis bald!'}
            ],
            'utter_iamabot': [
                {'text': 'Ich bin ein Bot, der von RASA erstellt wurde.'},
                {'text': 'Ich bin Ihr digitaler Assistent.'}
            ],
            'utter_happy': [
                {'text': 'Das freut mich zu hören!'},
                {'text': 'Wunderbar!'},
                {'text': 'Toll, dass es Ihnen gut geht!'}
            ],
            'utter_cheer_up': [
                {'text': 'Das tut mir leid. Kann ich Ihnen irgendwie helfen?'},
                {'text': 'Hier ist etwas, um Sie aufzumuntern:\\n🌟 Sie sind großartig! 🌟'}
            ],
            'utter_opening_hours': [
                {'text': 'Unsere Öffnungszeiten sind:\\nMontag-Freitag: 9:00-17:00\\nSamstag: 9:00-13:00\\nSonntag: geschlossen'}
            ],
            'utter_contact_info': [
                {'text': 'Sie können uns erreichen unter:\\n📞 Telefon: +49 123 456789\\n📧 Email: info@example.com\\n📍 Adresse: Musterstraße 1, 12345 Musterstadt'}
            ],
            'utter_services': [
                {'text': 'Wir bieten folgende Services an:\\n• Beratung\\n• Terminvereinbarung\\n• Informationen\\n• Kundenservice'}
            ],
            'utter_appointment_booked': [
                {'text': 'Ihr Termin wurde erfolgreich gebucht!'},
                {'text': 'Perfekt! Ihr Termin ist reserviert.'}
            ],
            'utter_appointment_cancelled': [
                {'text': 'Ihr Termin wurde storniert.'},
                {'text': 'Der Termin wurde erfolgreich abgesagt.'}
            ],
            'utter_ask_help': [
                {'text': 'Ich kann Ihnen bei folgenden Themen helfen:\\n• Öffnungszeiten\\n• Kontaktinformationen\\n• Terminbuchung\\n• Allgemeine Fragen'}
            ],
            'utter_default': [
                {'text': 'Entschuldigung, das habe ich nicht verstanden. Können Sie das anders formulieren?'},
                {'text': 'Das verstehe ich leider nicht. Können Sie mir mehr Details geben?'}
            ]
        },
        'actions': [
            'action_book_appointment',
            'action_cancel_appointment',
            'action_check_weather',
            'action_get_time'
        ],
        'session_config': {
            'session_expiration_time': 60,
            'carry_over_slots_to_new_session': True
        }
    }
    
    domain_path = rasa_dir / 'domain.yml'
    
    with open(domain_path, 'w', encoding='utf-8') as f:
        yaml.dump(domain_config, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f"✓ Created domain.yml with {len(domain_config['intents'])} intents")
    return domain_path

# Create project structure
rasa_dir = create_rasa_project_structure()
domain_path = create_domain_file(rasa_dir)

print(f"\n✓ RASA project structure created at: {rasa_dir}")

## Solution 2: Comprehensive NLU Training Data

In [ ]:
def create_nlu_training_data(rasa_dir):
    """Create comprehensive NLU training data."""
    
    nlu_data = {
        'version': '3.1',
        'nlu': [
            {
                'intent': 'greet',
                'examples': '''- hallo
- guten morgen
- guten tag
- hi
- hey
- servus
- moin
- schönen guten tag
- guten abend
- hallo zusammen
- tag
- na wie gehts
- hallo wie geht es ihnen
- einen schönen tag'''
            },
            {
                'intent': 'goodbye',
                'examples': '''- tschüss
- auf wiedersehen
- bis bald
- ciao
- bye
- bis später
- mach's gut
- schönen tag noch
- bis dann
- ade
- wir sehen uns
- einen schönen tag noch'''
            },
            {
                'intent': 'affirm',
                'examples': '''- ja
- genau
- richtig
- korrekt
- stimmt
- das ist richtig
- ja bitte
- ja gerne
- okay
- ok
- in ordnung
- perfect
- super
- einverstanden'''
            },
            {
                'intent': 'deny',
                'examples': '''- nein
- nein danke
- das stimmt nicht
- falsch
- das ist nicht richtig
- nö
- nope
- auf keinen fall
- niemals
- das möchte ich nicht
- das passt nicht'''
            },
            {
                'intent': 'mood_great',
                'examples': '''- perfekt
- sehr gut
- fantastisch
- wunderbar
- großartig
- ausgezeichnet
- toll
- super
- ich fühle mich großartig
- mir geht es sehr gut
- ich bin glücklich
- alles bestens'''
            },
            {
                'intent': 'mood_unhappy',
                'examples': '''- traurig
- unglücklich
- nicht so gut
- schlecht
- mir geht es nicht gut
- ich bin traurig
- das ist nicht schön
- frustriert
- verärgert
- enttäuscht
- ich bin unzufrieden'''
            },
            {
                'intent': 'bot_challenge',
                'examples': '''- bist du ein bot
- bist du menschlich
- spreche ich mit einem bot
- bist du echt
- wer bist du
- was bist du
- bist du eine ki
- bist du künstliche intelligenz
- bin ich mit einem menschen verbunden'''
            },
            {
                'intent': 'ask_opening_hours',
                'examples': '''- wann habt ihr geöffnet
- wie sind eure öffnungszeiten
- wann seid ihr offen
- öffnungszeiten bitte
- wann kann ich kommen
- wann habt ihr auf
- wie lange habt ihr geöffnet
- bis wann seid ihr da
- ab wann seid ihr geöffnet
- öffnungszeiten
- geöffnet heute
- habt ihr heute auf'''
            },
            {
                'intent': 'ask_contact_info',
                'examples': '''- wie kann ich euch erreichen
- kontakt informationen
- telefonnummer
- email adresse
- wo seid ihr
- adresse bitte
- kontaktdaten
- wie kann ich euch kontaktieren
- eure nummer
- wo findet ihr euch
- kontakt'''
            },
            {
                'intent': 'ask_services',
                'examples': '''- was bietet ihr an
- welche services habt ihr
- was kann ich bei euch machen
- eure dienstleistungen
- was macht ihr
- welche angebote habt ihr
- services
- leistungen
- angebot
- was gibt es bei euch
- was könnt ihr für mich tun'''
            },
            {
                'intent': 'book_appointment',
                'examples': '''- ich möchte einen termin buchen
- termin vereinbaren
- kann ich einen termin haben
- ich brauche einen termin
- termin bitte
- ich hätte gerne einen termin für [morgen](date)
- kann ich für [nächste woche](date) einen termin
- termin für [montag](date) um [14:00](time)
- ich möchte einen termin für [beratung](service_type)
- appointment booking
- termin machen'''
            },
            {
                'intent': 'cancel_appointment',
                'examples': '''- ich möchte meinen termin absagen
- termin stornieren
- termin absagen
- kann ich den termin canceln
- termin löschen
- ich kann nicht kommen
- termin verschieben
- appointment cancel
- stornierung'''
            },
            {
                'intent': 'ask_weather',
                'examples': '''- wie ist das wetter
- wetter heute
- regnet es
- wie warm ist es
- wetter
- scheint die sonne
- wetterbericht
- wie ist das wetter draußen'''
            },
            {
                'intent': 'ask_time',
                'examples': '''- wie spät ist es
- welche zeit haben wir
- uhrzeit bitte
- zeit
- what time is it
- current time'''
            },
            {
                'intent': 'ask_help',
                'examples': '''- hilfe
- kannst du mir helfen
- ich brauche hilfe
- help
- was kannst du
- wobei kannst du mir helfen
- unterstützung
- ich weiß nicht weiter
- kannst du mir sagen
- erklär mir das'''
            }
        ]
    }
    
    nlu_path = rasa_dir / 'data' / 'nlu.yml'
    
    with open(nlu_path, 'w', encoding='utf-8') as f:
        yaml.dump(nlu_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f"✓ Created NLU training data with {len(nlu_data['nlu'])} intents")
    
    # Count total examples
    total_examples = sum(len(intent['examples'].strip().split('\n')) for intent in nlu_data['nlu'])
    print(f"✓ Total training examples: {total_examples}")
    
    return nlu_path

def create_stories_data(rasa_dir):
    """Create comprehensive conversation stories."""
    
    stories_data = {
        'version': '3.1',
        'stories': [
            {
                'story': 'happy path',
                'steps': [
                    {'intent': 'greet'},
                    {'action': 'utter_greet'},
                    {'intent': 'mood_great'},
                    {'action': 'utter_happy'}
                ]
            },
            {
                'story': 'sad path 1',
                'steps': [
                    {'intent': 'greet'},
                    {'action': 'utter_greet'},
                    {'intent': 'mood_unhappy'},
                    {'action': 'utter_cheer_up'},
                    {'action': 'utter_ask_help'}
                ]
            },
            {
                'story': 'bot challenge',
                'steps': [
                    {'intent': 'bot_challenge'},
                    {'action': 'utter_iamabot'}
                ]
            },
            {
                'story': 'opening hours inquiry',
                'steps': [
                    {'intent': 'greet'},
                    {'action': 'utter_greet'},
                    {'intent': 'ask_opening_hours'},
                    {'action': 'utter_opening_hours'},
                    {'intent': 'goodbye'},
                    {'action': 'utter_goodbye'}
                ]
            },
            {
                'story': 'contact info inquiry',
                'steps': [
                    {'intent': 'ask_contact_info'},
                    {'action': 'utter_contact_info'}
                ]
            },
            {
                'story': 'services inquiry',
                'steps': [
                    {'intent': 'greet'},
                    {'action': 'utter_greet'},
                    {'intent': 'ask_services'},
                    {'action': 'utter_services'},
                    {'intent': 'affirm'},
                    {'action': 'utter_happy'}
                ]
            },
            {
                'story': 'appointment booking flow',
                'steps': [
                    {'intent': 'greet'},
                    {'action': 'utter_greet'},
                    {'intent': 'book_appointment'},
                    {'action': 'action_book_appointment'},
                    {'action': 'utter_appointment_booked'},
                    {'intent': 'goodbye'},
                    {'action': 'utter_goodbye'}
                ]
            },
            {
                'story': 'appointment cancellation',
                'steps': [
                    {'intent': 'cancel_appointment'},
                    {'action': 'action_cancel_appointment'},
                    {'action': 'utter_appointment_cancelled'}
                ]
            },
            {
                'story': 'weather inquiry',
                'steps': [
                    {'intent': 'ask_weather'},
                    {'action': 'action_check_weather'}
                ]
            },
            {
                'story': 'time inquiry',
                'steps': [
                    {'intent': 'ask_time'},
                    {'action': 'action_get_time'}
                ]
            },
            {
                'story': 'help request',
                'steps': [
                    {'intent': 'ask_help'},
                    {'action': 'utter_ask_help'}
                ]
            },
            {
                'story': 'interactive_story_1',
                'steps': [
                    {'intent': 'greet'},
                    {'action': 'utter_greet'},
                    {'intent': 'ask_opening_hours'},
                    {'action': 'utter_opening_hours'},
                    {'intent': 'book_appointment'},
                    {'action': 'action_book_appointment'},
                    {'action': 'utter_appointment_booked'},
                    {'intent': 'goodbye'},
                    {'action': 'utter_goodbye'}
                ]
            }
        ]
    }
    
    stories_path = rasa_dir / 'data' / 'stories.yml'
    
    with open(stories_path, 'w', encoding='utf-8') as f:
        yaml.dump(stories_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f"✓ Created stories with {len(stories_data['stories'])} conversation flows")
    return stories_path

# Create training data
nlu_path = create_nlu_training_data(rasa_dir)
stories_path = create_stories_data(rasa_dir)

## Solution 3: Configuration Files

In [ ]:
def create_config_file(rasa_dir):
    """Create RASA configuration file."""
    
    config_data = {
        'version': '3.1',
        'assistant_id': 'german_assistant_2024',
        'language': 'de',
        'pipeline': [
            {
                'name': 'WhitespaceTokenizer'
            },
            {
                'name': 'RegexFeaturizer'
            },
            {
                'name': 'LexicalSyntacticFeaturizer'
            },
            {
                'name': 'CountVectorsFeaturizer'
            },
            {
                'name': 'CountVectorsFeaturizer',
                'analyzer': 'char_wb',
                'min_ngram': 1,
                'max_ngram': 4
            },
            {
                'name': 'DIETClassifier',
                'epochs': 100,
                'constrain_similarities': True
            },
            {
                'name': 'EntitySynonymMapper'
            },
            {
                'name': 'ResponseSelector',
                'epochs': 100,
                'constrain_similarities': True
            },
            {
                'name': 'FallbackClassifier',
                'threshold': 0.3,
                'ambiguity_threshold': 0.1
            }
        ],
        'policies': [
            {
                'name': 'MemoizationPolicy'
            },
            {
                'name': 'RulePolicy'
            },
            {
                'name': 'UnexpecTEDIntentPolicy',
                'max_history': 5,
                'epochs': 100
            },
            {
                'name': 'TEDPolicy',
                'max_history': 5,
                'epochs': 100,
                'constrain_similarities': True
            }
        ]
    }
    
    config_path = rasa_dir / 'config.yml'
    
    with open(config_path, 'w', encoding='utf-8') as f:
        yaml.dump(config_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print("✓ Created config.yml with German language support")
    return config_path

def create_rules_file(rasa_dir):
    """Create rules for handling specific scenarios."""
    
    rules_data = {
        'version': '3.1',
        'rules': [
            {
                'rule': 'Say goodbye anytime the user says goodbye',
                'steps': [
                    {'intent': 'goodbye'},
                    {'action': 'utter_goodbye'}
                ]
            },
            {
                'rule': 'Say \'I am a bot\' anytime the user challenges',
                'steps': [
                    {'intent': 'bot_challenge'},
                    {'action': 'utter_iamabot'}
                ]
            },
            {
                'rule': 'Provide opening hours',
                'steps': [
                    {'intent': 'ask_opening_hours'},
                    {'action': 'utter_opening_hours'}
                ]
            },
            {
                'rule': 'Provide contact information',
                'steps': [
                    {'intent': 'ask_contact_info'},
                    {'action': 'utter_contact_info'}
                ]
            },
            {
                'rule': 'Provide services information',
                'steps': [
                    {'intent': 'ask_services'},
                    {'action': 'utter_services'}
                ]
            },
            {
                'rule': 'Handle help requests',
                'steps': [
                    {'intent': 'ask_help'},
                    {'action': 'utter_ask_help'}
                ]
            },
            {
                'rule': 'Activate fallback',
                'steps': [
                    {'intent': 'nlu_fallback'},
                    {'action': 'utter_default'}
                ]
            }
        ]
    }
    
    rules_path = rasa_dir / 'data' / 'rules.yml'
    
    with open(rules_path, 'w', encoding='utf-8') as f:
        yaml.dump(rules_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f"✓ Created rules.yml with {len(rules_data['rules'])} rules")
    return rules_path

def create_endpoints_file(rasa_dir):
    """Create endpoints configuration."""
    
    endpoints_data = {
        'action_endpoint': {
            'url': 'http://localhost:5055/webhook'
        },
        'tracker_store': {
            'type': 'InMemoryTrackerStore'
        }
    }
    
    endpoints_path = rasa_dir / 'endpoints.yml'
    
    with open(endpoints_path, 'w', encoding='utf-8') as f:
        yaml.dump(endpoints_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print("✓ Created endpoints.yml")
    return endpoints_path

# Create configuration files
config_path = create_config_file(rasa_dir)
rules_path = create_rules_file(rasa_dir)
endpoints_path = create_endpoints_file(rasa_dir)

## Solution 4: Custom Actions

In [ ]:
def create_custom_actions(rasa_dir):
    """Create custom actions for the RASA bot."""
    
    actions_code = '''"""
Custom actions for the German Assistant RASA bot.
"""

from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet
import datetime
import random


class ActionBookAppointment(Action):
    """Action to book an appointment."""

    def name(self) -> Text:
        return "action_book_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        # Get slot values
        appointment_date = tracker.get_slot("appointment_date")
        appointment_time = tracker.get_slot("appointment_time")
        service_type = tracker.get_slot("service_type")
        user_name = tracker.get_slot("user_name")

        # Default values if not provided
        if not appointment_date:
            appointment_date = "nächste Woche"
        if not appointment_time:
            appointment_time = "10:00"
        if not service_type:
            service_type = "Beratung"

        # Simulate booking process
        booking_id = f"APT-{random.randint(1000, 9999)}"
        
        message = f"""Ihr Termin wurde erfolgreich gebucht! 📅

📋 **Termindetails:**
• Datum: {appointment_date}
• Uhrzeit: {appointment_time}
• Service: {service_type}
• Buchungs-ID: {booking_id}

Wir freuen uns auf Ihren Besuch!"""

        dispatcher.utter_message(text=message)

        return [
            SlotSet("appointment_date", appointment_date),
            SlotSet("appointment_time", appointment_time),
            SlotSet("service_type", service_type)
        ]


class ActionCancelAppointment(Action):
    """Action to cancel an appointment."""

    def name(self) -> Text:
        return "action_cancel_appointment"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        messages = [
            "Ihr Termin wurde erfolgreich storniert. 🗑️",
            "Der Termin wurde abgesagt. Bei Fragen stehen wir gerne zur Verfügung.",
            "Stornierung erfolgreich. Sie können jederzeit einen neuen Termin buchen."
        ]
        
        dispatcher.utter_message(text=random.choice(messages))

        return [
            SlotSet("appointment_date", None),
            SlotSet("appointment_time", None),
            SlotSet("service_type", None)
        ]


class ActionCheckWeather(Action):
    """Action to provide weather information."""

    def name(self) -> Text:
        return "action_check_weather"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        # Simulate weather data
        weather_conditions = [
            "☀️ Sonnig, 22°C - Perfektes Wetter für einen Spaziergang!",
            "⛅ Teilweise bewölkt, 18°C - Angenehme Temperaturen.",
            "🌧️ Leichter Regen, 15°C - Vergessen Sie nicht den Regenschirm!",
            "❄️ Schnee, 2°C - Warme Kleidung empfohlen!",
            "🌤️ Wechselhaft, 20°C - Ein typischer deutscher Tag!"
        ]

        current_time = datetime.datetime.now()
        weather = random.choice(weather_conditions)
        
        message = f"""🌤️ **Aktuelles Wetter** ({current_time.strftime('%H:%M')})

{weather}

*Hinweis: Dies ist eine Simulation. Für aktuelle Wetterdaten besuchen Sie einen Wetterdienst.*"""

        dispatcher.utter_message(text=message)
        return []


class ActionGetTime(Action):
    """Action to provide current time."""

    def name(self) -> Text:
        return "action_get_time"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        current_time = datetime.datetime.now()
        
        # Format time in German style
        time_str = current_time.strftime('%H:%M')
        date_str = current_time.strftime('%d.%m.%Y')
        weekday = current_time.strftime('%A')
        
        # Translate weekday to German
        weekdays_de = {
            'Monday': 'Montag',
            'Tuesday': 'Dienstag', 
            'Wednesday': 'Mittwoch',
            'Thursday': 'Donnerstag',
            'Friday': 'Freitag',
            'Saturday': 'Samstag',
            'Sunday': 'Sonntag'
        }
        
        weekday_de = weekdays_de.get(weekday, weekday)
        
        message = f"""🕐 **Aktuelle Zeit**

⏰ {time_str} Uhr
📅 {weekday_de}, {date_str}"""

        dispatcher.utter_message(text=message)
        return []


class ActionDefaultFallback(Action):
    """Fallback action for unrecognized inputs."""

    def name(self) -> Text:
        return "action_default_fallback"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        fallback_messages = [
            "Entschuldigung, das habe ich nicht verstanden. 🤔\\nKönnen Sie das anders formulieren?",
            "Das verstehe ich leider nicht. 😅\\nWie kann ich Ihnen sonst helfen?",
            "Hmm, da bin ich überfragt. 🤷‍♀️\\nVersuchen Sie es mit anderen Worten?",
            "Das kenne ich noch nicht. 📚\\nFragen Sie nach Öffnungszeiten, Terminen oder Kontaktdaten!"
        ]

        dispatcher.utter_message(text=random.choice(fallback_messages))
        return []
'''

    actions_path = rasa_dir / 'actions' / 'actions.py'
    
    with open(actions_path, 'w', encoding='utf-8') as f:
        f.write(actions_code)
    
    # Create __init__.py
    init_path = rasa_dir / 'actions' / '__init__.py'
    with open(init_path, 'w') as f:
        f.write('# Actions module\\n')
    
    print("✓ Created custom actions in actions/actions.py")
    return actions_path

def create_credentials_file(rasa_dir):
    """Create credentials file for different channels."""
    
    credentials_data = {
        'rest': {},
        'socketio': {
            'user_message_evt': 'user_uttered',
            'bot_message_evt': 'bot_uttered',
            'session_persistence': True
        }
    }
    
    credentials_path = rasa_dir / 'credentials.yml'
    
    with open(credentials_path, 'w', encoding='utf-8') as f:
        yaml.dump(credentials_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print("✓ Created credentials.yml")
    return credentials_path

# Create custom actions and credentials
actions_path = create_custom_actions(rasa_dir)
credentials_path = create_credentials_file(rasa_dir)

## Solution 5: Testing and Validation

In [ ]:
def create_test_stories(rasa_dir):
    """Create test stories for evaluating the bot."""
    
    test_stories = {
        'version': '3.1',
        'stories': [
            {
                'story': 'test greeting and goodbye',
                'steps': [
                    {'user': 'hallo'},
                    {'action': 'utter_greet'},
                    {'user': 'tschüss'},
                    {'action': 'utter_goodbye'}
                ]
            },
            {
                'story': 'test opening hours',
                'steps': [
                    {'user': 'öffnungszeiten'},
                    {'action': 'utter_opening_hours'}
                ]
            },
            {
                'story': 'test appointment booking',
                'steps': [
                    {'user': 'ich möchte einen termin buchen'},
                    {'action': 'action_book_appointment'},
                    {'action': 'utter_appointment_booked'}
                ]
            },
            {
                'story': 'test bot challenge',
                'steps': [
                    {'user': 'bist du ein bot'},
                    {'action': 'utter_iamabot'}
                ]
            },
            {
                'story': 'test weather inquiry',
                'steps': [
                    {'user': 'wie ist das wetter'},
                    {'action': 'action_check_weather'}
                ]
            }
        ]
    }
    
    test_path = rasa_dir / 'tests' / 'test_stories.yml'
    
    with open(test_path, 'w', encoding='utf-8') as f:
        yaml.dump(test_stories, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f"✓ Created test stories with {len(test_stories['stories'])} test cases")
    return test_path

def validate_rasa_project(rasa_dir):
    """Validate the RASA project structure and files."""
    
    print("Validating RASA Project:")
    print("=" * 40)
    
    required_files = [
        'domain.yml',
        'config.yml',
        'endpoints.yml', 
        'credentials.yml',
        'data/nlu.yml',
        'data/stories.yml',
        'data/rules.yml',
        'actions/actions.py',
        'tests/test_stories.yml'
    ]
    
    validation_results = []
    
    for file_path in required_files:
        full_path = rasa_dir / file_path
        if full_path.exists():
            file_size = full_path.stat().st_size
            validation_results.append({
                'file': file_path,
                'exists': True,
                'size': file_size,
                'status': '✓'
            })
        else:
            validation_results.append({
                'file': file_path,
                'exists': False,
                'size': 0,
                'status': '✗'
            })
    
    # Print validation results
    print(f"{'File':<25} {'Status':<8} {'Size (bytes)':<12}")
    print("-" * 45)
    
    for result in validation_results:
        size_str = str(result['size']) if result['exists'] else 'N/A'
        print(f"{result['file']:<25} {result['status']:<8} {size_str:<12}")
    
    # Summary
    total_files = len(validation_results)
    valid_files = sum(1 for r in validation_results if r['exists'])
    
    print(f"\\nValidation Summary:")
    print(f"✓ Valid files: {valid_files}/{total_files}")
    
    if valid_files == total_files:
        print("🎉 All required files are present!")
        return True
    else:
        print(f"⚠️ Missing {total_files - valid_files} required files")
        return False

def create_dockerfile(rasa_dir):
    """Create Dockerfile for containerized deployment."""
    
    dockerfile_content = '''FROM rasa/rasa:3.6.4-full

# Copy project files
COPY . /app
WORKDIR /app

# Set user for security
USER 1001

# Train the model
RUN rasa train

# Expose ports
EXPOSE 5005

# Default command
CMD ["run", "--enable-api", "--cors", "*", "--debug"]
'''
    
    dockerfile_path = rasa_dir / 'Dockerfile'
    
    with open(dockerfile_path, 'w') as f:
        f.write(dockerfile_content)
    
    print("✓ Created Dockerfile for deployment")
    return dockerfile_path

def create_run_scripts(rasa_dir):
    """Create scripts to run the RASA bot."""
    
    # Training script
    train_script = '''#!/bin/bash
echo "Training RASA model..."
rasa train

if [ $? -eq 0 ]; then
    echo "✓ Training completed successfully!"
    echo "Model saved in models/ directory"
else
    echo "✗ Training failed!"
    exit 1
fi
'''
    
    # Run script
    run_script = '''#!/bin/bash
echo "Starting RASA server..."
echo "Bot will be available at http://localhost:5005"
echo "API documentation: http://localhost:5005/docs"
echo ""
echo "Press Ctrl+C to stop the server"
echo ""

rasa run --enable-api --cors "*" --debug
'''
    
    # Shell script
    shell_script = '''#!/bin/bash
echo "Starting RASA shell for interactive testing..."
echo "Type your messages and press Enter"
echo "Type '/stop' to exit"
echo ""

rasa shell
'''
    
    # Windows batch files
    train_bat = '''@echo off
echo Training RASA model...
rasa train

if %errorlevel% equ 0 (
    echo Training completed successfully!
    echo Model saved in models\\ directory
) else (
    echo Training failed!
    pause
    exit /b 1
)
pause
'''
    
    run_bat = '''@echo off
echo Starting RASA server...
echo Bot will be available at http://localhost:5005
echo API documentation: http://localhost:5005/docs
echo.
echo Press Ctrl+C to stop the server
echo.

rasa run --enable-api --cors "*" --debug
pause
'''
    
    # Create script files
    scripts = [
        ('train.sh', train_script),
        ('run.sh', run_script),
        ('shell.sh', shell_script),
        ('train.bat', train_bat),
        ('run.bat', run_bat)
    ]
    
    for script_name, script_content in scripts:
        script_path = rasa_dir / script_name
        with open(script_path, 'w') as f:
            f.write(script_content)
        
        # Make shell scripts executable (on Unix systems)
        if script_name.endswith('.sh'):
            try:
                os.chmod(script_path, 0o755)
            except:
                pass  # Ignore on Windows
    
    print("✓ Created run scripts (train.sh/bat, run.sh/bat, shell.sh)")
    return scripts

# Create test files and validation
test_path = create_test_stories(rasa_dir)
dockerfile_path = create_dockerfile(rasa_dir)
scripts = create_run_scripts(rasa_dir)

# Validate the complete project
is_valid = validate_rasa_project(rasa_dir)

## Solution 6: Advanced Features and Integration

In [ ]:
def create_forms_example(rasa_dir):
    """Create example forms for collecting user information."""
    
    # Add form to domain
    forms_domain_addition = '''
# Add this to your domain.yml file under forms:
forms:
  appointment_form:
    required_slots:
      - appointment_date
      - appointment_time
      - service_type
      - user_name

slots:
  appointment_date:
    type: text
    mappings:
    - type: from_entity
      entity: date
    - type: from_text
      conditions:
      - active_loop: appointment_form
        requested_slot: appointment_date
  
  appointment_time:
    type: text
    mappings:
    - type: from_entity
      entity: time
    - type: from_text
      conditions:
      - active_loop: appointment_form
        requested_slot: appointment_time
  
  service_type:
    type: text
    mappings:
    - type: from_entity
      entity: service_type
    - type: from_text
      conditions:
      - active_loop: appointment_form
        requested_slot: service_type
  
  user_name:
    type: text
    mappings:
    - type: from_entity
      entity: person
    - type: from_text
      conditions:
      - active_loop: appointment_form
        requested_slot: user_name

responses:
  utter_ask_appointment_date:
  - text: "Für welchen Tag möchten Sie den Termin? (z.B. morgen, nächste Woche, 15.12.2024)"
  
  utter_ask_appointment_time:
  - text: "Zu welcher Uhrzeit? (z.B. 10:00, 14:30)"
  
  utter_ask_service_type:
  - text: "Welche Art von Service benötigen Sie? (Beratung, Behandlung, Information)"
  
  utter_ask_user_name:
  - text: "Wie ist Ihr Name?"
  
  utter_submit:
  - text: "Danke! Ich verarbeite jetzt Ihre Terminanfrage..."
'''
    
    forms_path = rasa_dir / 'config' / 'forms_example.yml'
    
    with open(forms_path, 'w', encoding='utf-8') as f:
        f.write(forms_domain_addition)
    
    print("✓ Created forms example configuration")
    return forms_path

def create_integration_examples(rasa_dir):
    """Create examples for integrating with external services."""
    
    integration_code = '''"""
Integration examples for RASA bot with external services.
"""

import requests
import json
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher


class ActionWeatherAPI(Action):
    """Integrate with real weather API."""

    def name(self) -> Text:
        return "action_weather_api"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        # Example with OpenWeatherMap API (requires API key)
        api_key = "YOUR_API_KEY_HERE"
        city = "Berlin"  # Could be extracted from user input
        
        if api_key == "YOUR_API_KEY_HERE":
            # Fallback to mock data if no API key
            weather_info = "☀️ Sonnig, 22°C in Berlin (Mock-Daten)"
        else:
            try:
                url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric&lang=de"
                response = requests.get(url, timeout=5)
                data = response.json()
                
                temp = data['main']['temp']
                description = data['weather'][0]['description']
                weather_info = f"{description.title()}, {temp}°C in {city}"
                
            except Exception as e:
                weather_info = f"Wetterinformationen nicht verfügbar. Fehler: {str(e)}"

        dispatcher.utter_message(text=f"🌤️ Aktuelles Wetter: {weather_info}")
        return []


class ActionDatabaseLookup(Action):
    """Example for database integration."""

    def name(self) -> Text:
        return "action_database_lookup"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        user_name = tracker.get_slot("user_name")
        
        # Mock database lookup
        mock_database = {
            "Max Mustermann": {
                "customer_id": "C12345",
                "last_appointment": "15.11.2024",
                "preferred_time": "10:00"
            },
            "Anna Schmidt": {
                "customer_id": "C12346", 
                "last_appointment": "20.11.2024",
                "preferred_time": "14:00"
            }
        }
        
        if user_name and user_name in mock_database:
            customer_data = mock_database[user_name]
            message = f"""Hallo {user_name}! 👋
            
Ihre Kundendaten:
• Kunden-ID: {customer_data['customer_id']}
• Letzter Termin: {customer_data['last_appointment']}
• Bevorzugte Zeit: {customer_data['preferred_time']}"""
        else:
            message = "Kunde nicht in der Datenbank gefunden. Möchten Sie sich registrieren?"

        dispatcher.utter_message(text=message)
        return []


class ActionSendEmail(Action):
    """Example for email notifications."""

    def name(self) -> Text:
        return "action_send_email"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        user_name = tracker.get_slot("user_name")
        appointment_date = tracker.get_slot("appointment_date")
        appointment_time = tracker.get_slot("appointment_time")
        
        # Mock email sending (in real implementation, use SMTP or email service)
        email_content = f"""
Terminbestätigung für {user_name}

Datum: {appointment_date}
Uhrzeit: {appointment_time}

Mit freundlichen Grüßen,
Ihr Team
"""
        
        # In real implementation:
        # send_email(to="user@example.com", subject="Terminbestätigung", body=email_content)
        
        dispatcher.utter_message(text="📧 Bestätigungs-E-Mail wurde gesendet!")
        return []


class ActionLogInteraction(Action):
    """Log user interactions for analytics."""

    def name(self) -> Text:
        return "action_log_interaction"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        # Extract interaction data
        latest_message = tracker.latest_message
        intent = latest_message.get('intent', {}).get('name', 'unknown')
        confidence = latest_message.get('intent', {}).get('confidence', 0.0)
        entities = latest_message.get('entities', [])
        
        # Log data (in real implementation, send to analytics service)
        log_data = {
            "timestamp": tracker.latest_action_name,
            "user_id": tracker.sender_id,
            "intent": intent,
            "confidence": confidence,
            "entities": entities,
            "message_text": latest_message.get('text', '')
        }
        
        # Mock logging
        print(f"[LOG] User interaction: {json.dumps(log_data, indent=2)}")
        
        return []
'''
    
    integration_path = rasa_dir / 'actions' / 'integrations.py'
    
    with open(integration_path, 'w', encoding='utf-8') as f:
        f.write(integration_code)
    
    print("✓ Created integration examples")
    return integration_path

def create_deployment_guide(rasa_dir):
    """Create deployment guide and documentation."""
    
    deployment_guide = '''# RASA Bot Deployment Guide

## 🚀 Quick Start

### 1. Training the Model
```bash
# Linux/Mac
./train.sh

# Windows
train.bat

# Or directly
rasa train
```

### 2. Running the Bot
```bash
# Linux/Mac
./run.sh

# Windows  
run.bat

# Or directly
rasa run --enable-api --cors "*"
```

### 3. Interactive Testing
```bash
# Linux/Mac
./shell.sh

# Or directly
rasa shell
```

## 🔧 Configuration

### Environment Variables
```bash
export RASA_MODEL_PATH="./models"
export RASA_SERVER_PORT="5005"
export ACTION_SERVER_PORT="5055"
```

### Custom Actions Server
```bash
# Start actions server (in separate terminal)
rasa run actions

# Then start main server
rasa run --enable-api --cors "*"
```

## 🐳 Docker Deployment

### Build Image
```bash
docker build -t my-rasa-bot .
```

### Run Container
```bash
docker run -p 5005:5005 my-rasa-bot
```

### Docker Compose (with actions server)
```yaml
version: '3.8'
services:
  rasa:
    build: .
    ports:
      - "5005:5005"
    depends_on:
      - actions
    command: run --enable-api --cors "*" --endpoints endpoints.yml
  
  actions:
    build: .
    ports:
      - "5055:5055"
    command: run actions
```

## 🌐 Production Deployment

### 1. Cloud Platforms
- **Heroku**: Use `Procfile` and buildpacks
- **AWS**: ECS, Lambda, or EC2
- **Google Cloud**: Cloud Run or GKE
- **Azure**: Container Instances or AKS

### 2. Environment Setup
```bash
# Install production dependencies
pip install rasa[full]
pip install gunicorn

# Set production environment
export RASA_ENVIRONMENT=production
```

### 3. Monitoring and Logging
- Use structured logging
- Monitor model performance
- Track conversation metrics
- Set up alerts for failures

## 📊 Testing

### Unit Tests
```bash
rasa test nlu
rasa test core
```

### End-to-End Tests
```bash
rasa test --stories tests/test_stories.yml
```

### Performance Testing
```bash
# Test NLU accuracy
rasa test nlu --cross-validation

# Test core stories
rasa test core --stories data/stories.yml
```

## 🔐 Security

### API Security
- Use authentication tokens
- Implement rate limiting
- Validate input data
- Enable HTTPS in production

### Example nginx configuration:
```nginx
server {
    listen 443 ssl;
    server_name your-bot-domain.com;
    
    location / {
        proxy_pass http://localhost:5005;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

## 📈 Scaling

### Horizontal Scaling
- Load balance multiple RASA instances
- Use shared storage for models
- Implement session affinity if needed

### Vertical Scaling
- Increase memory for larger models
- Use GPU for faster inference
- Optimize model size

## 🛠️ Troubleshooting

### Common Issues
1. **Model not found**: Check model path
2. **Actions server not responding**: Verify endpoints.yml
3. **Low confidence scores**: Retrain with more data
4. **Memory issues**: Reduce model complexity

### Debug Mode
```bash
rasa run --debug --enable-api
```

### Logs Analysis
```bash
# View recent logs
tail -f rasa.log

# Search for errors
grep -i error rasa.log
```

## 📚 Additional Resources

- [RASA Documentation](https://rasa.com/docs)
- [Community Forum](https://forum.rasa.com)
- [GitHub Examples](https://github.com/RasaHQ/rasa)

## 🎯 Next Steps

1. **Improve NLU**: Add more training examples
2. **Enhance Conversations**: Create more complex stories
3. **Add Integrations**: Connect to databases, APIs
4. **Monitor Performance**: Set up analytics
5. **Scale Infrastructure**: Prepare for production load
'''

    guide_path = rasa_dir / 'DEPLOYMENT.md'
    
    with open(guide_path, 'w', encoding='utf-8') as f:
        f.write(deployment_guide)
    
    print("✓ Created deployment guide")
    return guide_path

def create_requirements_file(rasa_dir):
    """Create requirements.txt for Python dependencies."""
    
    requirements = '''rasa==3.6.4
rasa-sdk==3.6.1
spacy>=3.4.0,<3.8
https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0.tar.gz#egg=de_core_news_sm
requests>=2.28.0
pyyaml>=5.4.0
python-dateutil>=2.8.0
'''
    
    req_path = rasa_dir / 'requirements.txt'
    
    with open(req_path, 'w') as f:
        f.write(requirements)
    
    print("✓ Created requirements.txt")
    return req_path

# Create advanced features
forms_path = create_forms_example(rasa_dir)
integration_path = create_integration_examples(rasa_dir)
guide_path = create_deployment_guide(rasa_dir)
req_path = create_requirements_file(rasa_dir)

print("\n" + "="*50)
print("🎉 Complete RASA Chatbot Solution Created!")
print("="*50)

print(f"\n📁 Project Location: {rasa_dir}")
print("\n📋 What was created:")
print("• Complete project structure")
print("• Domain configuration with 15+ intents")
print("• NLU training data with 100+ examples")
print("• Conversation stories and rules")
print("• Custom actions with real functionality")
print("• Test cases and validation")
print("• Docker configuration")
print("• Run scripts for different platforms")
print("• Integration examples")
print("• Deployment documentation")

print("\n🚀 Getting Started:")
print("1. Navigate to the project directory:")
print(f"   cd {rasa_dir}")
print("2. Install RASA (if not already installed):")
print("   pip install rasa")
print("3. Train the model:")
print("   rasa train")
print("4. Test in shell:")
print("   rasa shell")
print("5. Run as server:")
print("   rasa run --enable-api --cors '*'")

print("\n💡 Features included:")
print("• German language support")
print("• Appointment booking system")
print("• Weather information")
print("• Time queries")
print("• Fallback handling")
print("• Custom actions")
print("• Integration examples")
print("• Production deployment ready")